In [1]:
import numpy as np
import pandas as pd
import sys

# tweepy
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream

# pymongo
import config
import ppp
import pymongo
from pymongo import MongoClient
import dnspython
import urllib.parse
import regex
import re

In [2]:
# Twitter API
access_token = config.access_token
access_token_secret = config.access_token_secret
consumer_key = config.consumer_key
consumer_secret = config.consumer_secret

## SQLITE

In [6]:
import sqlite3

In [7]:
conn = sqlite3.connect('test.db')
c = conn.cursor()

In [13]:
c.execute("create table coffee (main_id varchar(40), id varchar(20), name varchar(50), followers integer null, following integer null)")

OperationalError: table coffee already exists

## JSON to SQLite

In [14]:
import json
with open("data/test-out (2)", "r") as f1:
    for line in f1:
        try:
            data = json.loads(line)
            c.execute("insert into coffee values (?, ?, ?, ?, ?)",
                      [data['id_str'], data['user']['id'], data['user']['screen_name'], data['user']['followers_count'], data['user']['friends_count']])
        except:
            continue

In [15]:
cursor = conn.execute("select * from coffee where main_id == '1253319701473587209'")
rows = cursor.fetchall()

In [16]:
for row in rows:
    print(row)

('1253319701473587209', '1227184861850394626', 'kristiavalenzu1', 85, 65)
('1253319701473587209', '1227184861850394626', 'kristiavalenzu1', 85, 65)


## JSON to MongoDB

In [3]:
# setting mongodb config
username = urllib.parse.quote_plus(ppp.username)
password = urllib.parse.quote_plus(ppp.password)
cluster = MongoClient('mongodb+srv://%s:%s@cluster0-tlu5n.mongodb.net/test?retryWrites=true&w=majority' % (username, password))

In [4]:
# defining cluster and collection
db = cluster["coffee_twitter"]
collection = db["tweets"]

In [11]:
# reading json and pushing tweet contents to MongoDB database
with open("data/test-out (2)", "r") as f1:
    for line in f1:
        try:
            data = json.loads(line)
            idx = data['id_str']
            full_tweet = data['retweeted_status']['extended_tweet']['full_text']
            lang = data['lang']
            retweeted = data['retweeted']
            reply_count = data['reply_count']
            retweet_count = data['retweet_count']
            
            post = {"id": idx, "full_tweet": full_tweet, "lang": lang, "retweeted": retweeted, "reply_count": reply_count,
                    "retweet_count": retweet_count}
            
            collection.insert_one(post)
        except:
            continue

## Find Information

In [6]:
# query
results = collection.find({"id": "1253317430220337152"})
print(results)

In [7]:
for r in results:
    print(r['full_tweet'] + "\n***")

Man, this quarantine has changed a lot.  My eating is better, my alcohol intake is at all-time lows as of a week or so ago (see: none).  BUT... my black coffee intake is at an all-time high.
***


In [50]:
# query text
results = collection.find( { "$text": { "$search": "starbucks" } } )
print(results)

In [46]:
# query exact phrase
results = collection.find( { "$text": { "$search": "\"black rifle\"" } } )
print(results)

In [52]:
# query term exclusion
results = collection.find( { "$text": { "$search": "-black sugar" } } )
print(results)

In [137]:
# query
results = collection.find({"lang": "fr"})
print(results)

In [139]:
for r in results:
    print(r['full_tweet'] + "\n***")

•"Je crois parfois que Dieu, en créant l'homme, a quelque peu surestimé ses capacités."
•Oscar Wilde

Le café est prêt
***
Retrouvez les @lumberjack dans leur émission Coffee Break qui commence à 15H par une session feedback où vous ne devez pas hésiter à leur envoyer vos démos : feedback@coffeebreak.bar 🤩 
Puis retrouvez en live sur notre page FB leurs interviews de DJs et DJanes ! 🤫❤️ https://t.co/nCroE2n4cB
***
Retrouvez les @lumberjack dans leur émission Coffee Break qui commence à 15H par une session feedback où vous ne devez pas hésiter à leur envoyer vos démos : feedback@coffeebreak.bar 🤩 
Puis retrouvez en live sur notre page FB leurs interviews de DJs et DJanes ! 🤫❤️ https://t.co/nCroE2n4cB
***


In [74]:
# extract start with "hey" regex
results = collection.find( { "full_tweet": { "$regex": "^hey" } } )
print(results)

In [75]:
for r in results:
    print(r['full_tweet'])

hey guys, so if you add milk/creamer/sugar/etc to coffee do they count as condiments because you add them to cover to give it flavour? if so, is coffee also a condiment because you add it to water to give it flavour?
hey guys, so if you add milk/creamer/sugar/etc to coffee do they count as condiments because you add them to cover to give it flavour? if so, is coffee also a condiment because you add it to water to give it flavour?
hey guys, so if you add milk/creamer/sugar/etc to coffee do they count as condiments because you add them to cover to give it flavour? if so, is coffee also a condiment because you add it to water to give it flavour?
hey guys, so if you add milk/creamer/sugar/etc to coffee do they count as condiments because you add them to cover to give it flavour? if so, is coffee also a condiment because you add it to water to give it flavour?


In [149]:
def find_tweet_hashtags(hasht = None, user = None, find = 'hash'):
    if hasht is not None:
        look = "((#" + hasht + "))"
    if user is not None:
        look = "((@" + user + "))"
    results = collection.find( { "full_tweet": { "$regex": look } } )
    
    for r in results:
        hashtag = r['full_tweet']
        if find == 'hash':
            x = re.findall(r"(#\w+)", hashtag)
        if find == 'user':
            x = re.findall(r"(@\w+)", hashtag)
        if find == 'full_tweet':
            x = hashtag
        print(x)


In [150]:
find_tweet_hashtags(hasht = "dalgona", find = 'hash')

['#dalgonacoffeechallenge', '#Dalgonacoffee', '#FluffyCoffee', '#coffee', '#whippedcoffee', '#Louisiana', '#Acadiana', '#Lafayette']
['#dalgonacoffeechallenge', '#Dalgonacoffee', '#FluffyCoffee', '#coffee', '#whippedcoffee', '#Louisiana', '#Acadiana', '#Lafayette']
['#dalgonacoffeechallenge', '#Dalgonacoffee', '#FluffyCoffee', '#coffee', '#whippedcoffee', '#Louisiana', '#Acadiana', '#Lafayette']
['#dalgonacoffeechallenge', '#Dalgonacoffee', '#FluffyCoffee', '#coffee', '#whippedcoffee', '#Louisiana', '#Acadiana', '#Lafayette']


In [151]:
find_tweet_hashtags(user = "coffee", find = 'user')

['@lumberjack', '@coffeebreak']
['@lumberjack', '@coffeebreak']


In [152]:
find_tweet_hashtags(hasht = "java", find = 'full_tweet')

What do you believe to be the number one requirement to be a successful developer? 🤔

i am doubt full about the answer being mostly coffee🙄😂☕

#100DaysOfCode #html #IoT #CodeNewbie #javascript #react #301DaysOfCode #GameDev
What do you believe to be the number one requirement to be a successful developer? 🤔

i am doubt full about the answer being mostly coffee🙄😂☕

#100DaysOfCode #html #IoT #CodeNewbie #javascript #react #301DaysOfCode #GameDev
What do you believe to be the number one requirement to be a successful developer? 🤔

i am doubt full about the answer being mostly coffee🙄😂☕

#100DaysOfCode #html #IoT #CodeNewbie #javascript #react #301DaysOfCode #GameDev
What do you believe to be the number one requirement to be a successful developer? 🤔

i am doubt full about the answer being mostly coffee🙄😂☕

#100DaysOfCode #html #IoT #CodeNewbie #javascript #react #301DaysOfCode #GameDev
What do you believe to be the number one requirement to be a successful developer? 🤔

i am doubt full ab

### Mongo + SQLite function

In [206]:
def find_by_word(word = None, tweet = None):

    results = collection.find({ "$text": { "$search": word } })
    
    idx = []
    lang = []
    user_id = []
    full_text = []
    for r in results:
        idx.append(r['id'])
        lang.append(r['lang'])
        full_text.append(r['full_tweet'])
        
        #sql
        cursor = conn.execute("select name from coffee where main_id == {}".format(r['id']))
        rows = cursor.fetchall()
        
        for row in rows:
            user_id.append(row)
    
    for i, val in enumerate((idx)):
        print(idx[i], ":::", lang[i], ":::", user_id[i])
        if tweet is not None:
            print(":::", full_text[i], "***\n")

In [207]:
find_by_word(word = 'chai')

1253338334488756224 ::: in ::: ('Hanniballad',)
1253337426048585729 ::: in ::: ('Hanniballad',)
1253336524176756738 ::: in ::: ('Mbusih',)
1253336073985380357 ::: in ::: ('Mbusih',)
1253335602537193474 ::: in ::: ('florswaggle',)
1253338334488756224 ::: in ::: ('florswaggle',)
1253337426048585729 ::: in ::: ('Ddaaante',)
1253336524176756738 ::: in ::: ('Ddaaante',)
1253336073985380357 ::: in ::: ('AishaWanjiku_',)
1253335602537193474 ::: in ::: ('AishaWanjiku_',)
1253327849966104576 ::: in ::: ('Hanniballad',)
1253327849966104576 ::: in ::: ('Hanniballad',)


In [208]:
find_by_word(word = 'java', tweet = True)

1253330996243558407 ::: en ::: ('bigevilbeard',)
::: How about some java with your java this morning? ☕️ Learn to code, get inspired and connect with #softwaredevelopers through @CiscoDevNet: https://t.co/3CAKsrR64Y #CiscoCert https://t.co/oiMoKQcLut ***

1253330996243558407 ::: en ::: ('bigevilbeard',)
::: How about some java with your java this morning? ☕️ Learn to code, get inspired and connect with #softwaredevelopers through @CiscoDevNet: https://t.co/3CAKsrR64Y #CiscoCert https://t.co/oiMoKQcLut ***

1253317504069500929 ::: en ::: ('bigevilbeard',)
::: UPCOMING "BEERS"
FRI: Flashlight Mango Hard Seltzer
MON: Flashlight Key Lime Hard Seltzer w/ Lemon &amp; Coconut
Next FRI? Terra Java Coffee Baltic Porter
2 weeks out: Scioto Coast Double IPA
In the Tank: Southern Terminus Mexican Lager
In the Tank: Beta Flash NE IPA (Batch #23) https://t.co/ysHztgMocB ***



In [195]:
# query text
results = collection.find( { "$text": { "$search": "java" } }, {"score": {"$meta": "textScore"} } ).sort([('score', {"$meta": 'textScore'})])
for r in results:
    print(r['score'], "\n")

0.8250000000000001 

0.8250000000000001 

0.5108695652173914 



In [186]:
def info_by_relevancy(text = None, sort_by_score = None):


    results = collection.find({ "$text": { "$search": text } }, {"score": {"$meta": "textScore"} }).sort([('score', {"$meta": 'textScore'})])
    
    idx = []
    lang = []
    user_id = []
    full_text = []
    score = []
    for r in results:
        idx.append(r['id'])
        lang.append(r['lang'])
        full_text.append(r['full_tweet'])
        score.append(r['score'])
        
        #sql
        cursor = conn.execute("select name from coffee where main_id == {}".format(r['id']))
        rows = cursor.fetchall()
        
        for row in rows:
            user_id.append(row)
    
    user_id = list(set(user_id))
    full_text = list(set(full_text))
    score = list(set(score))
    
    for i, val in enumerate(user_id):
        print("User --> ", user_id[i], "\nTweet --> ", full_text[i], "\nScore -->" , score[i], "\n\n")

In [187]:
info_by_relevancy(text = 'dunkin', sort_by_score = True)

User -->  ('drzewieckinbh',) 
Tweet -->  By @hooks_ciara  Café Busy Bean has signed a contract with the City to move into the former Dunkin Donuts location next to the police station at 135 Main St.

 https://t.co/EtRVa0o5cC 
Score --> 0.5333333333333333 


User -->  ('MissyJo79',) 
Tweet -->  gotta give the people what they want.
here is the official Dunkin coffee flavor ranking:
1) peppermint mocha
2) banana split
3) thin mint
4) blueberry mocha (trust me)
5) cinnamon mocha
6) caramel mocha
7) pumpkin
8) irish creme 
9) mint chocolate chip
10) oreo 
Score --> 0.5227272727272727 


User -->  ('TBen112',) 
Tweet -->  RETWEET &amp; WIN!! - COFFEE WEDNESDAY!

RETWEET + LIKE THIS POST &amp; BE ENTERED TO WIN 1 of 2 $25 Tim Hortons or Dunkin Donuts Custom Jimmy Freight Gift Cards!

*
JIMMY FREIGHT- BOOK FULL &amp; CLOSED.  For Limited Syndicate Shares call @WinbakFarm  905-838-2145 https://t.co/cIxGf50fK5 
Score --> 0.5128205128205128 


User -->  ('Rodneyd11196564',) 
Tweet -->  @GodFamil

In [144]:
def find_user_lang(lang = None):
    # query
    results = collection.find({"lang": lang})
    
    idx = []
    full_tweet = []
    user_id = []
    for r in results:
        full_tweet.append(r['full_tweet'])
        idx.append(r['id'])
        
        #sql
        cursor = conn.execute("select name from coffee where main_id == {}".format(r['id']))
        rows = cursor.fetchall()
        
        for row in rows:
            user_id.append(row)
    
    user_id = list(set(user_id))
    full_tweet = list(set(full_tweet))
    
    for i, val in enumerate(user_id):
        print(user_id[i], "->  ", full_tweet[i])

In [145]:
find_user_lang(lang = 'fr')

('Becike_Claude',) ->   •"Je crois parfois que Dieu, en créant l'homme, a quelque peu surestimé ses capacités."
•Oscar Wilde

Le café est prêt
('Remaille91',) ->   Retrouvez les @lumberjack dans leur émission Coffee Break qui commence à 15H par une session feedback où vous ne devez pas hésiter à leur envoyer vos démos : feedback@coffeebreak.bar 🤩 
Puis retrouvez en live sur notre page FB leurs interviews de DJs et DJanes ! 🤫❤️ https://t.co/nCroE2n4cB


## find df

In [202]:
def find_by_word_df(word = None, tweet = None):

    results = collection.find({ "$text": { "$search": word } }, {"score": {"$meta": "textScore"} })
    
    idx = []
    lang = []
    user_id = []
    full_text = []
    user_tweet = []
    followers = []
    following = []
    score = []
    
    for r in results:
        idx.append(r['id'])
        lang.append(r['lang'])
        full_text.append(r['full_tweet'])
        score.append(r['score'])
        #sql
        cursor = conn.execute("select name, followers, following from coffee where main_id == {}".format(r['id']))
        rows = cursor.fetchall()
        
        for row in rows:
            user_id.append(row[0])
            followers.append(row[1])
            following.append(row[2])
    
    user_id = list(set(user_id))
    full_text = list(set(full_text))
    followers = list(set(followers))
    following = list(set(following))
    score = list(set(score))
    
    for i, val in enumerate(user_id):
        len_tweet = (len(full_text[i].split(' ')))
        user_tweet.append([user_id[i], followers[i], following[i], full_text[i],re.findall(r'(#\w+)', full_text[i]),  
                          re.findall(r'(@\w+)', full_text[i]), len_tweet, score[i]])
        
#    user_tweet = pd.DataFrame({'user_id': user_id, 'tweet': full_text})
    
    return(pd.DataFrame(user_tweet, columns = ['user', 'followers', 'following', 'tweet', 'hashtags', 'mentions', 
                                               'len_tweet', 'relevancy_score']))
    #return(set(user_id), set(full_text))

In [203]:
x = find_by_word_df(word = 'dunkin')

In [204]:
x

,user,followers,following,tweet,hashtags,mentions,len_tweet,relevancy_score
0,TBen112,32322,76,By @hooks_ciara Café Busy Bean has signed a c...,[],[@hooks_ciara],31,0.533333
1,Rodneyd11196564,1351,789,RETWEET &amp; WIN!! - COFFEE WEDNESDAY!\n\nRET...,[],[@WinbakFarm],44,0.512195
2,MissyJo79,170,4440,gotta give the people what they want.\nhere is...,[],[],36,0.512821
3,kelseykruk,619,35517,@GodFamilyJesus Dunkin. I'm boycotting Starbuc...,[#MadeInChina],[@GodFamilyJesus],21,0.522727
4,drzewieckinbh,80,1214,Let’s raise a cup to all the employees around ...,[],[],36,0.520833


In [17]:
cursor = conn.execute("select * from coffee where main_id == '1253319701473587209'")
rows = cursor.fetchall()

In [22]:
for row in rows:
    print(row)

('1253319701473587209', '1227184861850394626', 'kristiavalenzu1', 85, 65)
('1253319701473587209', '1227184861850394626', 'kristiavalenzu1', 85, 65)
